# On the purpose of analyzing x and scaling/normalizing to make GNN run with this x

In [2]:
import numpy as np

In [ ]:
# load numpy array
load_x = np.loadtxt("data/descriptors_x.csv",delimiter = ",")